In [2]:
'''
작성일 : 2020-09-10
작성자 : 정성모
코드 개요 :
    Arima model을 사용하여 데이터 예측하여 실제 데이터와 차이 확인.
    getrq로 정렬하여 데이터 index 재생성​, 원 데이터에서 complete – nvme_sq 값이 0인 row를
    확인하고 해당 row에​ nvme_sq - getrq를 0 값으로 변경​
결과 : mae : 0.0002
'''

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from numpy import nan
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA
%matplotlib inline
import time

def main():
    data = pd.read_csv("C:/Users/jsm97/Desktop/raw193.csv")
    data = data[data['block_getrq']!=0]
    data = data.sort_values(['block_getrq'],ascending="False")
    data = data.reset_index(drop=True)
    
    # complete의 값이 0인 index를 찾아 nvme_sq - getrq에도 0 값 할당
    nv_get = pd.read_csv("C:/Users/jsm97/Desktop/nv_get.csv")
    nv_get = nv_get['0']
    nv_get = nv_get[0:50000]
    
    data = data['nvme_sq']-data['block_getrq']
 
    # 결측치 부분 찾기
    zero_data = nv_get[nv_get==0]
    zero_data = zero_data.reset_index()
    zero_data['index_Difference'] = zero_data['index']
    zero_data['index_Difference'] = zero_data['index_Difference'] - zero_data['index_Difference'].shift(1,fill_value=0)
    zero_temp = zero_data[zero_data['index_Difference']>1]
    zero_temp = zero_temp.reset_index()
    print(zero_temp)
    print(zero_data)
    
    zero_temp['level_0'] = zero_temp['level_0'].shift(-1) - zero_temp['level_0']
    nan_value = zero_data.iloc[-1]['index'] - zero_temp.iloc[-1]['index'] + 1
    zero_temp = zero_temp.replace(nan,nan_value)
    zero_temp['level_0'] = zero_temp['level_0']
    #print(zero_temp)
    #print(len(zero_temp))
    
    # 예측 할 데이터 갯수
    len_pred = 1
    # 학습 할 데이터 갯수
    len_train = 50
    
    total = np.zeros((len(zero_data),3))
    j = 0
    # 결측치 부분 ARIMA 모델을 이용하여 예측
    for i in range(len(zero_temp)):
        test = zero_temp.iloc[i]['level_0'].astype('int64')
        train = zero_temp.iloc[i]['index'].astype('int64')
        #plt.figure(figsize=(15,15))
        #plt.scatter(range(train-len_train,train),nv_get[train-len_train:train],s=1)
        #print("test :",test)
        #print("train :",train)
        if test > len_pred:
            while test >= len_pred:
                if train < len_train:
                    data_temp = nv_get[train-train:train]
                    nv_get[train:train+len_pred] = My_arima(data_temp,train,len_pred)
                    total_mae, original, forecast = mae(nv_get[train:train+len_pred], data[train:train+len_pred])
                    total[j] = original, forecast, total_mae
                    train = train + len_pred
                    test = test - len_pred
                    j += 1
                    #print("part0")
                    #print(data_temp)
                    
                else:
                    data_temp = nv_get[train-len_train:train]
                    nv_get[train:train+len_pred] = My_arima(data_temp,train,len_pred)
                    total_mae, original, forecast = mae(nv_get[train:train+len_pred], data[train:train+len_pred])
                    total[j] = original, forecast, total_mae
                    test = test - len_pred
                    train = train + len_pred
                    j += 1
                    #print("part1")
                    #print(data_temp)

        else:
            if train < len_train:
                data_temp = nv_get[train - train:train]
                nv_get[train:train+len_pred] = My_arima(data_temp,train,len_pred)
                total_mae, original, forecast = mae(nv_get[train:train+len_pred], data[train:train+len_pred])
                total[j] = original, forecast, total_mae
                j += 1
                #print("part2")
            else:
                data_temp = nv_get[train - len_train:train]
                nv_get[train:train+len_pred] = My_arima(data_temp,train,len_pred)
                total_mae, original, forecast = mae(nv_get[train:train+len_pred], data[train:train+len_pred])
                total[j] = original, forecast, total_mae
                j += 1
                #print("part3")
    
    #print(zero_temp)
    #print(zero_data)
    #print(data)
    print("total : ", total)
    # total = original, forecast, mae 
    total = pd.DataFrame(total)
    total.to_csv("C:/Users/jsm97/Desktop/ARIMA_diff_evaluation_50.csv")
    

def mae(forecast,original):
    '''
    함수 개요 :
        원 데이터와 예측 데이터의 절대값 오차 평균 (mae)
    파라미터 :
        forecast = 예측 데이터
        original = 원 데이터
    '''
    forecast = np.array(forecast).reshape(-1,1)
    original = np.array(original).reshape(-1,1)
    error_mean = np.mean(np.abs(original - forecast), axis=0)
    #print("original : ",original)
    #print("forecast : ",forecast)
    #print("mae : ",error_mean)
    return error_mean, original, forecast
    
def My_arima(data_temp,train,test):
    '''
    함수 개요 :
        ARIMA Model로 DATA의 대한 ARIMA 최적의 파라미터를 찾고, 결측치를 예측
    파라미터 :
        data_temp = 학습 할 데이터
        train = 학습 할 데이터 index 번호
        test = 예측 할 데이터 갯수
    '''
    # P-value Check
    #result = adfuller(data_temp)
    #print(f'원 데이터 ADF Statistic: {result[0]:.3f}')
    #print(f'원 데이터 p-value: {result[1]:.3f}')
    #data_temp = data_temp.diff().dropna()
    #reslut = adfuller(data_temp)
    #print(f'1차 차분 ADF Statistic: {result[0]:.3f}')
    #print(f'1차 차분 p-value: {result[1]:.3f}')
    
    # ARIMA Parameter Select
    list_arima = my_auto_arima(data_temp, [3,3,3])
    
    model = ARIMA(data_temp, order=(int(list_arima[0]),int(list_arima[1]),int(list_arima[2])))
    model_fit = model.fit()
    #print(model_fit.summary())
    forecast = model_fit.forecast(steps=test)[0]
    forecast = pd.Series(forecast)
    
    forecast.index = list(range(train,train+test))
    #print("forecast")
    #test_data = data1[train:train+test]
    #print("Original")
    #print(test_data)
    #data[train:train+test] = forecast
    return forecast
    #print("input to data")
    #print(data[train:train+test])

    # plot
    #plt.figure(figsize=(15,15))
    #plt.subplot(5,1,1)
    #plt.title("Original Data")
    #plt.plot(test_data)
    #plt.subplot(5,1,2)
    #plt.title("Forecast Data")
    #plt.plot(forecast)
    #plt.subplot(5,1,3)
    #plt.plot(test_data)
    #plt.plot(forecast)

def my_auto_arima(data, order,sort = 'AIC'):
    '''
    함수 개요 :
        data의 대한 ARIMA 모델의 파라미터들을 계산하여 최적의 값 출력
    파라미터 :
        data = 학습 할 데이터
        order = AR,I,MA의 각 파라미터의 범위
    '''
    import re
    
    order_list = []
    aic_list = []
    bic_lsit = []
    for p in range(order[0]):
        for d in range(order[1]):
            for q in range(order[2]):
                model = ARIMA(data, order=(p,d,q))
                try:
                    model_fit = model.fit()
                    c_order = f'p{p} d{d} q{q}'
                    aic = model_fit.aic
                    #bic = model_fit.bic
                    order_list.append(c_order)
                    aic_list.append(aic)
                    #bic_list.append(bic)
                except:
                    pass
    result_df = pd.DataFrame(list(zip(order_list, aic_list)),columns=['order','AIC'])
    result_df.sort_values(sort, inplace=True)
    str_data = result_df.iloc[0]['order']
    numbers = re.findall('\d+',str_data)
    return numbers
main()

     level_0  index    0  index_Difference
0          0   2563  0.0              2563
1          1   2570  0.0                 7
2          4   2582  0.0                10
3          6   3045  0.0               462
4          7   3120  0.0                75
..       ...    ...  ...               ...
585      711  49813  0.0                 3
586      712  49819  0.0                 6
587      714  49823  0.0                 3
588      716  49864  0.0                40
589      717  49893  0.0                29

[590 rows x 4 columns]
     index    0  index_Difference
0     2563  0.0              2563
1     2570  0.0                 7
2     2571  0.0                 1
3     2572  0.0                 1
4     2582  0.0                10
..     ...  ...               ...
713  49820  0.0                 1
714  49823  0.0                 3
715  49824  0.0                 1
716  49864  0.0                40
717  49893  0.0                29

[718 rows x 3 columns]


C:\Users\jsm97\anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\jsm97\anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\jsm97\anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\jsm97\anaconda3\lib\site-packages\statsmodels\base\model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\Users\jsm97\anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retval

C:\Users\jsm97\anaconda3\lib\site-packages\statsmodels\base\model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\Users\jsm97\anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\jsm97\anaconda3\lib\site-packages\statsmodels\base\model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\Users\jsm97\anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\jsm97\anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", Converg

C:\Users\jsm97\anaconda3\lib\site-packages\statsmodels\base\model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\Users\jsm97\anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\jsm97\anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\jsm97\anaconda3\lib\site-packages\statsmodels\base\model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\Users\jsm97\anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", Converg

C:\Users\jsm97\anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\jsm97\anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\jsm97\anaconda3\lib\site-packages\statsmodels\base\model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\Users\jsm97\anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\jsm97\anaconda3\lib\site-packages\statsmodels\base\model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInver

...

C:\Users\jsm97\anaconda3\lib\site-packages\statsmodels\base\model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\Users\jsm97\anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\jsm97\anaconda3\lib\site-packages\statsmodels\base\model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\Users\jsm97\anaconda3\lib\site-packages\statsmodels\base\model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\Users\jsm97\anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning

C:\Users\jsm97\anaconda3\lib\site-packages\statsmodels\base\model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\Users\jsm97\anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\jsm97\anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\jsm97\anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\jsm97\anaconda3\lib\site-packages\statsmodels\base\model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInver

C:\Users\jsm97\anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\jsm97\anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\jsm97\anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\jsm97\anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\jsm97\anaconda3\lib\site-packages\statsmodels\base\model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', H

total :  [[8.07109755e-05 5.14728042e-05 2.92381712e-05]
 [9.04826447e-06 1.15752275e-04 1.06704010e-04]
 [5.09219244e-05 9.17132274e-05 4.07913030e-05]
 ...
 [8.44399910e-05 7.83482402e-05 6.09175080e-06]
 [8.69277865e-05 8.52817698e-05 1.64601664e-06]
 [9.08649527e-05 1.06103557e-04 1.52386042e-05]]


C:\Users\jsm97\anaconda3\lib\site-packages\statsmodels\base\model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\Users\jsm97\anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
